In [1]:
import parastell.parastell as ps
import parastell.magnet_coils as magnet_coils
import parastell.invessel_build as ivb
import numpy as np
import src.pystell.read_vmec as read_vmec
from parastell.cubit_io import init_cubit
import cubit

init_cubit()

def shrink_matrix(mat, column_step, row_step):
    """
    removes rows and columns from a matrix,
    keeps first and last columns and rows.
    # of columns and rows should be evenly divisible by the step size,
    respectively
    """
    if len(mat.shape) == 1:
        mat = np.hstack((mat, mat[-1]))
        mat = mat[::column_step]
        return mat
    else:
        mat = np.hstack((mat, mat[:, -1].reshape(-1, 1)))
        mat = np.vstack((mat, mat[-1, :]))
        mat = mat[:, ::column_step]
        mat = mat[::row_step]
        return mat


[2024-06-26 20:42:07.635] [info] Found python version 3 at /opt/Coreform-Cubit-2023.11/bin/python3/libcubit_python3.so
[2024-06-26 20:42:07.635] [info] Looking for Python
[2024-06-26 20:42:07.635] [info] Version 3 interpreter found
[2024-06-26 20:42:07.638] [info] RLM session initialized


QObject::startTimer: Timers can only be used with threads started with QThread
QObject::startTimer: Timers can only be used with threads started with QThread
QObject::startTimer: Timers can only be used with threads started with QThread
QObject::startTimer: Timers can only be used with threads started with QThread
QObject::startTimer: Timers can only be used with threads started with QThread


Loading command plugins from /opt/Coreform-Cubit-2023.11/bin/plugins
Loaded Svalinn plugin.
-- DAGMC export command available.
-- MCNP import command available.


In [2]:


coils_file = 'coils_wistelld.txt'
circ_cross_section = ['circle', 25]
toroidal_extent = 90.0

coils = magnet_coils.MagnetSet(coils_file, circ_cross_section, toroidal_extent)
coils._extract_filaments()
coils._set_average_radial_distance()
coils._set_filtered_filaments()

filtered_filaments = coils.filtered_filaments

coils.build_coil_surface()



[[1090.19744173  648.10306047   -3.48054498]
 [1084.54105656  646.11602031    5.13982361]
 [1077.90108477  644.54179036   13.94808739]
 ...
 [1098.02342925  653.32219556  -20.26253828]
 [1094.72823786  650.50925454  -11.93962402]
 [1090.19744173  648.10306047   -3.48054498]]
[[1090.19744173  648.10306047   -3.48054498]
 [1084.54105656  646.11602031    5.13982361]
 [1077.90108477  644.54179036   13.94808739]
 ...
 [1098.02342925  653.32219556  -20.26253828]
 [1094.72823786  650.50925454  -11.93962402]
 [1090.19744173  648.10306047   -3.48054498]]


In [3]:


# Define directory to export all output files to
export_dir = ''
# Define plasma equilibrium VMEC file
vmec_file = 'plasma_wistelld.nc'

vmec = read_vmec.VMECData(vmec_file)

# Define build parameters for in-vessel components
toroidal_angles = np.linspace(0, 90, 72)
poloidal_angles = np.linspace(0,360,96)
wall_s = 1.08

# Define a matrix of uniform unit thickness
uniform_unit_thickness = np.ones((len(toroidal_angles), len(poloidal_angles)))

radial_build_dict = {}

radial_build = ivb.RadialBuild(toroidal_angles, poloidal_angles, wall_s, radial_build_dict)
build = ivb.InVesselBuild(vmec, radial_build, split_chamber = False)
build.populate_surfaces()
build.calculate_loci()
build._poloidal_angles_exp.shape
build._toroidal_angles_exp.shape

14:38:26: Constructing radial build...
14:38:26: Populating surface objects for in-vessel components...
14:38:29: Computing point cloud for in-vessel components...


(72,)

In [4]:
loci = build.get_loci()[0]
ribs = build.Surfaces['chamber'].Ribs
len(ribs)

72

In [5]:
distances = []
for rib in ribs:
    print(rib)
    distance_subset = []
    for point, direction in zip(rib.rib_loci, rib._normals()):
        cubit.cmd(f"create vertex {point[0]} {point[1]} {point[2]}")
        vertex_id = max(cubit.get_entities("vertex"))
        cubit.cmd(f'create curve location at vertex {vertex_id} location fire ray location at vertex {vertex_id} direction {direction[0]} {direction[1]} {direction[2]} at surface all maximum hits 1')
        curve_id = max(cubit.get_entities('curve'))
        distance = cubit.get_curve_length(curve_id)
        distance_subset.append(distance)
    distances.append(distance_subset)

In [6]:
cubit.cmd('save cub5 "point_cloud.cub5" overwrite')

True

In [8]:
### now for every loci I need to determine the direction to offset in
### do the raytracing
### store the distance in a matrix the same shape as the one with the vertexes in it

In [9]:
thickness_matrix = np.array(distances)
print(thickness_matrix.shape)
shrunk_matrix = shrink_matrix(thickness_matrix, 2, 2)
shrunk_matrix.shape

(72, 96)


(37, 49)

In [10]:
thickness_matrix.shape
# weird that this shape doesn't match

import pandas as pd 
df = pd.DataFrame(shrunk_matrix)
df.to_csv('data2.csv')

min(thickness_matrix.flatten())

79.44030909451543

In [11]:
num_phi = thickness_matrix.shape[0]
num_theta = thickness_matrix.shape[1]

toroidal_angles = shrink_matrix(np.rad2deg(build._toroidal_angles_exp), 2,2)
poloidal_angles = shrink_matrix(np.rad2deg(build._poloidal_angles_exp), 2,2)

print(toroidal_angles)
print(poloidal_angles)


[ 0.          2.53521127  5.07042254  7.6056338  10.14084507 12.67605634
 15.21126761 17.74647887 20.28169014 22.81690141 25.35211268 27.88732394
 30.42253521 32.95774648 35.49295775 38.02816901 40.56338028 43.09859155
 45.63380282 48.16901408 50.70422535 53.23943662 55.77464789 58.30985915
 60.84507042 63.38028169 65.91549296 68.45070423 70.98591549 73.52112676
 76.05633803 78.5915493  81.12676056 83.66197183 86.1971831  88.73239437
 90.        ]
[  0.           7.57894737  15.15789474  22.73684211  30.31578947
  37.89473684  45.47368421  53.05263158  60.63157895  68.21052632
  75.78947368  83.36842105  90.94736842  98.52631579 106.10526316
 113.68421053 121.26315789 128.84210526 136.42105263 144.
 151.57894737 159.15789474 166.73684211 174.31578947 181.89473684
 189.47368421 197.05263158 204.63157895 212.21052632 219.78947368
 227.36842105 234.94736842 242.52631579 250.10526316 257.68421053
 265.26315789 272.84210526 280.42105263 288.         295.57894737
 303.15789474 310.73684211 3

: 

In [12]:

# Instantiate ParaStell build
stellarator = ps.Stellarator(vmec_file)

radial_build_dict = {
    'radial_real_estate': {
        'thickness_matrix': shrunk_matrix
    }
}

stellarator.construct_invessel_build(
    toroidal_angles,
    poloidal_angles,
    wall_s,
    radial_build_dict,
    num_ribs = num_phi,
    num_rib_pts = num_theta,

)

stellarator.export_invessel_build(
    export_cad_to_dagmc=False,
    export_dir=export_dir
)

14:29:03: Constructing radial build...
14:29:03: Populating surface objects for in-vessel components...
14:29:11: Computing point cloud for in-vessel components...
14:29:11: Constructing CadQuery objects for in-vessel components...


In [ ]:
ribs = stellarator.invessel_build.Surfaces['radial_real_estate'].Ribs

In [ ]:
cubit.cmd('reset')

True

In [ ]:
for rib in ribs:
    print(rib)
    distance_subset = []
    for point, direction in zip(rib.rib_loci, rib._normals()):
        cubit.cmd(f"create vertex {point[0]} {point[1]} {point[2]}")
        vertex_id = max(cubit.get_entities("vertex"))

In [ ]:
cubit.cmd('save cub5 "point_cloud.cub5" overwrite')

True